In [ ]:
import numpy as np
from sympy import symbols, simplify
import sympy
one = sympy.Rational(1)
from BSeries import bs
h = sympy.Symbol('h')

In [ ]:
p, q = symbols('p,q')
u = [p,q]
f = np.array([-u[1]/(u[0]**2+u[1]**2), u[0]/(u[0]**2+u[1]**2)])
IC = [1.,0.]
simplify(f)

In [ ]:
# Runge's Method
A = np.array([[0,0],[one/2,0]])
b = np.array([0,one])


series = bs.modified_equation(u, f, A, b, order=7)
series = simplify(series)

In [ ]:
series

In [ ]:
term0=simplify(series[0])
numer0 = term0.as_numer_denom()[0]
denom0 = sympy.factor(term0.as_numer_denom()[1])
term1=simplify(series[1])
numer1 = term1.as_numer_denom()[0]
denom1 = term1.as_numer_denom()[1]

In [ ]:
test = sympy.Poly(numer0,h)
coeffs = test.all_coeffs()[::-1]
expr = 0
for j, coeff in enumerate(coeffs):
    display(h**j*sympy.factor(coeff)/denom0)
    expr += h**j*sympy.factor(coeff)/denom0

In [ ]:
expr

In [ ]:
simplify(expr - term0)

In [ ]:
rhs = []
for term in series:
    numer, denom = term.as_numer_denom()
    denom = sympy.factor(denom)
    P = sympy.Poly(numer,h)
    coeffs = P.all_coeffs()[::-1]
    expr = 0
    for j, coeff in enumerate(coeffs):
        expr += h**j*sympy.factor(coeff)/denom0
    rhs.append(expr)
    
rhs = simplify(np.array(rhs))
simplify(rhs-series)

In [ ]:
rhs